In [1]:
# -*- coding: utf8 -*-
import os
import time
import numpy as np
import math as m
import openturns as ot
import json
import requests

In [2]:
from evaluate import evaluate

In [3]:
username = 'testuser'
password = 'testpass'
set_id = -1
problem_id  = 2
def g_fun(x):
        #g_val_comp = evaluate(username, password, set_id, problem_id, x)
        g_val_sys, g_val_comp, msg = evaluate(username, password, set_id, problem_id, x)
        return g_val_comp

In [4]:
x=[[0.545, 1.23],[ 0.7, 1],[0.545, 1.23]]

In [5]:
start_time = time.time()
print(g_fun(x))
interval = time.time() - start_time
print ("Durée de la simulation :", round(interval, 2), "s")

[1.2918078899383545, 1.3069183826446533, 1.2918078899383545]
Durée de la simulation : 0.87 s


In [6]:
myFunction = ot.PythonFunction(2, 1, g_fun)

In [7]:
start_time = time.time()
print(myFunction(x))
interval = time.time() - start_time
print ("Durée de la simulation :", round(interval, 2), "s")

    [ y0      ]
0 : [ 1.29181 ]
1 : [ 1.30692 ]
2 : [ 1.29181 ]
Durée de la simulation : 2.31 s


In [8]:
dist_X1 = ot.Normal(0., 1.)
dist_X2 = ot.Normal(0., 1.)
myDistribution = ot.ComposedDistribution([dist_X1, dist_X2])
myRandomVector = ot.RandomVector(myDistribution)

In [9]:
myFunction = ot.PythonFunction(2, 1, g_fun)
myOutputVector = ot.CompositeRandomVector(myFunction, myRandomVector)
#myOutputVector.getSample(10)

In [10]:
event = ot.Event(myOutputVector, ot.LessOrEqual(), 0.0)
# We create an OptimizationAlgorithm algorithm
solver = ot.SQP()
algo = ot.SORM(solver, event, [0.,0.])
algo.run()
result = algo.getResult()

In [11]:
print(result.getEventProbabilityBreitung(),
result.getEventProbabilityHohenBichler(),
result.getEventProbabilityTvedt())

0.004403034626735833 0.004268174603760942 0.004208097382029138


In [12]:
optres = result.getOptimizationResult()
print(
"Design point:", optres.getOptimalPoint(),"\n" 
"Absolute error:", optres.getAbsoluteError(),"\n"#norme inifnie entre Xn et Xn+1
"Residual error:", optres.getResidualError() ,"\n" 
"Evaluation number:", optres.getEvaluationNumber())

Design point: [1.76777,1.76777] 
Absolute error: 2.396972138859378e-07 
Residual error: 2.3969721404294707e-07 
Evaluation number: 7
